In [2]:
import sys

import numpy as np
import pandas as pd
import datetime
import time

# music api imports
import spotipy
import spotipy.util as util
import billboard
import pygn
import pylast

import requests
import re
from bs4 import BeautifulSoup
from time import sleep
import pickle

In [4]:
# timing function
def timefunc(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        print f.__name__, 'took', end - start, 'seconds'
        return result
    return f_timer

In [5]:
# Setting up tokens and authorization - hidden for github, use own tokens

# Spotify
token = util.prompt_for_user_token(<INSERT SPOTIFY USERNAME>,
                                   client_id=<INSERT SPOTIFY CLIENT ID>,
                                   client_secret=<INSERT SPOTIFY CLIENT SECRET>,
                                   redirect_uri=<INSERT REDIRECT URI>)

spotify = spotipy.Spotify(auth=token)

# Gracenote
clientid=<INSERT GRACENOTE CLIENTID>
userid = pygn.register(clientid)

# last.fm
apikey = <INSERT LASTFM API KEY>
apisecret = <INSERT LASTFM API SECRET>
username = <INSERT YOUR USERNAME>
password_hash = pylast.md5(<INSERT YOUR PASSWORD>)

lastfm = pylast.LastFMNetwork(api_key = apikey, api_secret = apisecret,
                               username = username, password_hash = password_hash)

In [4]:
# not used - script to tie billboard charting info with spotify audio features
@timefunc
def scrape_song_info(chart,start_date,end_date):
   
    '''Scrape weekly chart data and spotify information for each track between specified dates.  
    Returns a dataframe with chart contents for each week and audio features for each track'''

    dates = [datetime.datetime.strptime(end_date, '%Y-%m-%d')-datetime.timedelta(7)]
    while min(dates)-datetime.timedelta(7)>=datetime.datetime.strptime(start_date, '%Y-%m-%d'):
        dates.append(dates[-1]-datetime.timedelta(7))

    charts_data = {}
    for i in dates:
        search_date = i.strftime(format='%Y-%m-%d')
        week = billboard.ChartData(chart, date=search_date, fetch=True, quantize=True).date
        charts = billboard.ChartData(chart, date=search_date, fetch=True, quantize=True)
        charts_data[week] = charts

    charts_df_data = []

    for k,v in charts_data.iteritems():
        for i in v:
            df_data = {}
            df_data['week'] = k
            df_data['rank'] = i.rank
            df_data['title'] = i.title
            df_data['artist'] = i.artist
            df_data['chart'] = chart           
            charts_df_data.append(df_data)

    charts_df = pd.DataFrame(charts_df_data)

    charting_songs = {}
    ID_set = []

    for v in charts_data.values():
        for i in v:
            title = i.title
            artist = i.artist
            ID = i.spotifyID
            if ID not in ID_set:
                ID_set.append(ID)
                charting_songs[(title,artist)] = ID

    spotify = spotipy.Spotify(auth=token)
    spotify_info = {k: spotify.audio_features(tracks=[v]) for k,v in charting_songs.iteritems()}

    spot_info = []

    for k,v in spotify_info.iteritems():
        try:
            spot_dict = {}
            spot_dict = v[0]
            spot_dict['title'] = k[0]
            spot_dict['artist'] = k[1]
            spot_info.append(spot_dict)
        except:
            pass

    spotify_info = pd.DataFrame(spot_info)
    spotify_info = spotify_info.drop(['analysis_url','track_href','type','uri'],axis=1)

    return charts_df,spotify_info

In [5]:
charts_df, spotify_info = scrape_song_info('hot-100', '2016-08-30', '2016-09-01')

scrape_song_info took 9.31413388252 seconds


In [10]:
# Examples of gracenote fuzzy match & full records
song1 = pygn.search(clientid, userid, artist='Zara', track='Lush')
song2 = pygn.search(clientid, userid, artist='The Chainsmokers', track='Closer')

song1

{'album_art_url': 'http://akamai-b.cdn.cddbp.net/cds/2.0/cover/2F8A/B566/9458/E56D_medium_front.jpg?cid=1157132724',
 'album_artist_name': 'Zara Larsson',
 'album_gnid': '528962250-88888A804C41AA5C7598C43C29F47FF0',
 'album_title': 'Lush Life [Single]',
 'album_year': '2015',
 'artist_bio_url': '',
 'artist_era': {'1': {'ID': '29483', 'TEXT': '2000s'},
  '2': {'ID': '29493', 'TEXT': 'Late 2000s'}},
 'artist_image_url': '',
 'artist_origin': {'1': {'ID': '29896', 'TEXT': 'Scandinavia'},
  '2': {'ID': '29991', 'TEXT': 'Sweden'},
  '4': {'ID': '30894', 'TEXT': 'Stockholm'}},
 'artist_type': {'1': {'ID': '29423', 'TEXT': 'Female'},
  '2': {'ID': '29443', 'TEXT': 'Female'}},
 'genre': {'1': {'ID': '35469', 'TEXT': 'Pop'},
  '2': {'ID': '35493', 'TEXT': 'Western Pop'},
  '3': {'ID': '25637', 'TEXT': 'Dance Pop'}},
 'mood': {'1': {'ID': '65330', 'TEXT': 'Rowdy'},
  '2': {'ID': '43035', 'TEXT': 'Driving Dark Groove'}},
 'radio_id': '',
 'review_url': '',
 'tempo': {'1': {'ID': '34283', 'TEXT':

In [6]:
moods = {'Pastoral / Serene':[0,0], 'Delicate / Tranquil':[0,1], 'Hopeful / Breezy':[0,2], 'Cheerful / Playful':[0,3], 'Carefree Pop':[0,4], 'Party / Fun':[0,5], 'Showy / Rousing':[0,6], 'Lusty / Jaunty':[0,7], 'Loud Celebratory':[0,8], 'Euphoric Energy':[0,9], 
         'Reverent / Healing':[1,0], 'Quiet / Introspective':[1,1], 'Friendly':[1,2], 'Charming / Easygoing':[1,3], 'Soulful / Easygoing':[1,4], 'Happy / Soulful':[1,5], 'Playful / Swingin':[1,6], 'Exuberant / Festive':[1,7], 'Upbeat Pop Groove':[1,8], 'Happy Excitement':[1,9], 
         'Refined / Mannered':[2,0], 'Awakening / Stately':[2,1], 'Sweet / Sincere':[2,2], 'Heartfelt Passion':[2,3], 'Strong / Stable':[2,4], 'Powerful / Heroic':[2,5], 'Invigorating / Joyous':[2,6], 'Jubilant / Soulful':[2,7], 'Ramshackle / Rollicking':[2,8], 'Wild / Rowdy':[2,9], 
         'Romantic / Lyrical':[3,0], 'Light Groovy':[3,1], 'Dramatic / Romantic':[3,2], 'Lush / Romantic':[3,3], 'Dramatic Emotion':[3,4], 'Idealistic / Stirring':[3,5], 'Focused Sparkling':[3,6], 'Triumphant / Rousing':[3,7], 'Confident / Tough':[3,8], 'Driving Dark Groove':[3,9],
         'Tender / Sincere':[4,0], 'Gentle Bittersweet':[4,1], 'Suave / Sultry':[4,2], 'Dark Playful':[4,3], 'Soft Soulful':[4,4], 'Sensual Groove':[4,5], 'Dark Sparkling Lyrical':[4,6], 'Fiery Groove':[4,7], 'Arousing Groove':[4,8], 'Heavy Beat':[4,9], 
         'Lyrical Sentimental':[5,0], 'Cool Melancholy':[5,1], 'Intimate Bittersweet':[5,2], 'Smoky / Romantic':[5,3], 'Dreamy Pulse':[5,4], 'Intimate Passionate':[5,5], 'Rhythm Energetic': [5,6], 'Abstract Groove':[5,7], 'Edgy / Sexy':[5,8], 'Abstract Beat':[5,9], 
         'Mysterious / Dreamy':[6,0], 'Light Melancholy':[6,1], 'Casual Groove':[6,2], 'Wary / Defiant':[6,3], 'Bittersweet Pop':[6,4], 'Energetic Yearning':[6,5], 'Dark Pop':[6,6], 'Dark Pop Intensity':[6,7], 'Heavy Brooding':[6,8], 'Hard Positive Excitement':[6,9], 
         'Wistful / Forlorn':[7,0], 'Sad / Soulful':[7,1], 'Cool Confidence':[7,2], 'Dark Groovy':[7,3], 'Sensitive / Exploring':[7,4], 'Energetic Dreamy':[7,5], 'Dark Urgent':[7,6], 'Energetic Anxious':[7,7], 'Attitude / Defiant':[7,8], 'Hard Dark Excitement':[7,9], 
         'Solemn / Spiritual':[8,0], 'Enigmatic / Mysterious':[8,1], 'Sober / Determined':[8,2], 'Strumming Yearning':[8,3], 'Melodramatic':[8,4], 'Hypnotic Rhythm':[8,5], 'Evocative / Intriguing':[8,6], 'Energetic Melancholy':[8,7], 'Dark Hard Beat':[8,8], 'Heavy Triumphant':[8,9], 
         'Dark Cosmic':[9,0], 'Creepy / Ominous':[9,1], 'Depressed / Lonely':[9,2], 'Gritty / Soulful':[9,3], 'Serious / Cerebral':[9,4], 'Thrilling':[9,5], 'Dreamy Brooding':[9,6], 'Alienated / Brooding':[9,7], 'Chaotic / Intense':[9,8], 'Aggressive Power':[9,9]}

## Get Spotify Song Information

In [23]:
@timefunc
# get categories 
def get_categories():
    category_ids = []
    for i in spotify.categories(limit = 50)['categories']['items']:
        category_ids.append(i.get('id'))
    return category_ids

@timefunc
# get playlists from list of categories
def get_playlists(categories):
    playlist_ids = []
    for i in categories:
        for j in spotify.category_playlists(i, limit = 50)['playlists']['items']:
            playlist_ids.append(j.get('id'))
    return playlist_ids

# get song ids from list of playlist ids
@timefunc
def get_songs(playlists):
    song_ids = []
    for i in playlists:
        try:
            for j in spotify.user_playlist('spotify', i)['tracks']['items']:
                song_ids.append(j['track']['id'])
        except:
            pass
        time.sleep(.1)
    return song_ids

In [16]:
categories = get_categories()
playlists = get_playlists(categories)
songs = get_songs(playlists)

get_categories took 0.0767250061035 seconds
get_playlists took 7.64370298386 seconds
get_songs took 2210.98003101 seconds


In [29]:
songs_df = pd.DataFrame(list(set(songs)))

In [67]:
print len(playlists)
print len(songs_df)

764
26079


In [8]:
songs_df.head()

,artist,explicit,id,popularity,title
0,Nirvana,False,3FUsMXBxA4V7eUwQ7B0HQO,48,Love Buzz
1,The Presidents Of The United States Of America,False,2JdzB67NvIa90K4gEZPLeX,58,Lump
2,Stanton Warriors,False,0zXLMZUCYZCx8Bq4jCr174,30,So Sweet
3,Natalie Marchenko,False,14w7n7gi635VCb5f7t2OQ5,37,License To Chill - Pacha Sax Lounge RMX
4,Robyn,False,01NJd6s7Kyn6NSsPp503sh,28,Be Mine! - Live At The Cherrytree House


## Load Pickle Point

In [32]:
songs_df.to_pickle('songs_df.pkl')

In [7]:
songs_df = pd.read_pickle('data/songs_df.pkl')

In [60]:
songs_df.columns = ['id']

In [69]:
def chunks(l, n):
    n = max(1, n)
    return [l[i:i+n] for i in range(0, len(l), n)]

In [191]:
@timefunc
def complete_songs_df(df):
    spotify_title_artists = []

    for j in chunks(list(df.id),50):
        try:
            for i in spotify.tracks(j)['tracks']:
                try:
                    title_artist = {}
                    primary_artist = i['artists'][0]['name'].encode('ascii', 'ignore')
                    name = i['name'].encode('ascii', 'ignore')
                    popularity = i['popularity']
                    explicit = i['explicit']
                    ID = i['id']
                    
                    title_artist['id'] = ID
                    title_artist['title'] = name
                    title_artist['artist'] = primary_artist
                    title_artist['popularity'] = popularity
                    title_artist['explicit'] = explicit
                    spotify_title_artists.append(title_artist)
                except:
                    pass
        except:
            pass
    
    return spotify_title_artists

In [192]:
songs_plus = complete_songs_df(songs_df)

complete_songs_df took 144.593239069 seconds


In [194]:
songs_df_plus = pd.DataFrame(songs_plus)

In [204]:
len(songs_df_plus)

26029

### Load Pickle Point

In [196]:
songs_df_plus.to_pickle('data/songs_df.pkl')

In [12]:
songs_df_plus = pd.read_pickle('data/songs_df.pkl')

## Get Audio Features

In [103]:
@timefunc
def complete_audio_features(df):
    features_list = []
    for j in chunks(list(df.id),50):
        features_add = spotify.audio_features(tracks=j)
        features_list.extend(features_add)
    return features_list

In [104]:
audio_features = complete_audio_features(songs_df_plus)

retrying ...1secs
complete_audio_features took 171.850511074 seconds


In [145]:
def make_audio_features_df(audio_features):
    audio_features_df = pd.DataFrame(columns = audio_features[0])
    for i in audio_features:
        audio_features_df = audio_features_df.append(i, ignore_index=True)
    return audio_features_df

In [ ]:
audio_features_df = make_audio_features(audio_features)

### Load Pickle Point

In [147]:
audio_features_df.to_pickle('data/audio_features.pkl')

In [11]:
audio_features_df = pd.read_pickle('data/audio_features.pkl')

## Merge Song and Audio Feature Dataframes

In [197]:
songs = pd.merge(songs_df_plus, audio_features_df, on='id')

In [ ]:
songs.drop(['track_href', 'analysis_url', 'uri', 'type'], axis=1, inplace=True)

In [10]:
songs.head(10)

,artist,explicit,id,popularity,title,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,key,duration_ms,loudness,valence,mode
0,Nirvana,False,3FUsMXBxA4V7eUwQ7B0HQO,48,Love Buzz,0.885,0.169,138.153,0.0354,0.000032,0.196,4,0.47,3.0,215120.0,-7.224,0.78,1.0
1,The Presidents Of The United States Of America,False,2JdzB67NvIa90K4gEZPLeX,58,Lump,0.873,0.165,142.726,0.0386,0.004680,0,4,0.499,6.0,134200.0,-3.902,0.869,1.0
2,Stanton Warriors,False,0zXLMZUCYZCx8Bq4jCr174,30,So Sweet,0.936,0.118,127.997,0.0425,0.000531,0.594,4,0.715,4.0,201337.0,-6.127,0.106,0.0
3,Natalie Marchenko,False,14w7n7gi635VCb5f7t2OQ5,37,License To Chill - Pacha Sax Lounge RMX,0.511,0.24,171.963,0.0302,0.262000,0.837,4,0.581,5.0,317811.0,-8.199,0.399,0.0
4,Robyn,False,01NJd6s7Kyn6NSsPp503sh,28,Be Mine! - Live At The Cherrytree House,0.249,0.114,116.599,0.0451,0.955000,0,4,0.533,0.0,272053.0,-8.532,0.513,0.0
5,Pearl Jam,False,6qgpO647NPBUvBkm56vV7d,5,Black,0.548,0.179,76.020,0.0287,0.294000,0,4,0.376,4.0,343667.0,-10.580,0.233,0.0
6,Sam Hunt,False,3wx2kQWPn9p5UppQbNhPAk,67,Leave The Night On,0.953,0.349,171.971,0.0624,0.099600,0,4,0.516,9.0,192160.0,-3.828,0.849,1.0
7,George Carlin,False,5DuxYjcuebFPontojL1NEE,0,War Pictures,0.546,0.147,72.940,0.961,0.921000,0,3,0.611,7.0,222200.0,-10.444,0.491,1.0
8,Kaya May,False,4eg3wsiTrPa7KpwVai2VY7,56,"Hands to Myself, Love Yourself, Perfect (Acous...",0.445,0.113,110.056,0.05,0.057500,0,4,0.702,9.0,226893.0,-9.693,0.326,0.0
9,Notaker,False,341ShRaeysVdxLetlI391K,56,Infinite,0.669,0.108,99.987,0.0483,0.093700,0.82,4,0.633,1.0,355200.0,-8.124,0.0384,0.0


### Load Pickle Point

In [234]:
songs.to_pickle('spotify_songs.pkl')

In [9]:
songs = pd.read_pickle('data/spotify_songs.pkl')